# Imports

In [ ]:
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import imutils
import math as mth
import os

from google.colab.patches import cv2_imshow
from skimage.io import imread_collection
from scipy.spatial import distance as dist
from os import listdir

######## HOG ########################
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
#######################################

# Load images query and database

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

In [ ]:
# Mettre le chemin du dossier contenant les images requêtes avec le dossier database à l'intérieur
dir_query = 'drive/MyDrive/Colab Notebooks/TP1_data'
dir_data  = 'drive/MyDrive/Colab Notebooks/TP1_data/database'

name_query = list()
name_data  = list()

################# read images name   #########################
for images in os.listdir(dir_query):
    if (images.endswith(".jpg")):
        name_query.append(images)
        
name_query.sort()


for images in os.listdir(dir_data):
    if (images.endswith(".jpg")):
        name_data.append(images)

name_data.sort()        
###############################################################


In [ ]:
# Vérification du chemin
os.chdir(dir_query)
cwd = os.getcwd()
print(cwd)

In [ ]:
################## Read images into a dict ###########################
images_query = {}
images_data  = {}

# dir_query = '*jpg'
for name in name_query:
    tmp = cv.imread(name)
    img = cv.cvtColor(tmp, cv.COLOR_BGR2RGB)
    images_query[name] = img


dir_data = 'database/'
for name in name_data:
    tmp = cv.imread((dir_data + name))
    img = cv.cvtColor(tmp, cv.COLOR_BGR2RGB)
    images_data[name] = img

# col_data = imread_collection(dir_data, as_gray = True)

######################################################################

In [ ]:
# plt.imshow(images_query['cat_query.jpg'])

# Images modification functions

In [ ]:
# Source :
# https://www.pyimagesearch.com/2021/02/03/opencv-image-translation/
# ------------------------------------------------------------------
def ImageTranslation(image):
    T = np.float32([[1,0,50], 
                   [0,1,50]])

    shiftedImage = cv.warpAffine(image, T, (image.shape[1], image.shape[0]))

    return shiftedImage

In [ ]:
# Source :
# https://www.programmerall.com/article/10721831285/
# ------------------------------------------------------------------
def ImageGaussianNoise(image, mean, var):
    
    image = np.array(image/255, dtype=float)

    noise = np.random.normal(mean, var ** 0.5, image.shape)

    imageNoisy = image + noise

    imageNoisy = np.clip(imageNoisy, 0, 1)

    imageNoisy = np.uint8(imageNoisy*255)
    
    return imageNoisy

In [ ]:
# Source :
# https://www.pyimagesearch.com/2021/01/20/opencv-rotate-image/
# ------------------------------------------------------------------
def ImageRotation(image, angle):    
    # grab the dimensions of the image and calculate the center of the
    # image
    # (h, w) = image.shape[:2]
    # (cX, cY) = (w // 2, h // 2)

    # rotate our image by 45 degrees around the center of the image
    # RotationMatrix = cv.getRotationMatrix2D((cX, cY), angle, 1.0)
    
    # imageRotated = cv.warpAffine(image, RotationMatrix, (w, h))

    imageRotated = imutils.rotate_bound(image, angle)

    return imageRotated

In [ ]:
def ImageGrayScaled(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY) 

In [ ]:
plt.figure()
# plt.imshow(ImageGaussianNoise(images_query['airplane_query.jpg'], 0, 1))
# plt.imshow(ImageRotation(images_query['car_query.jpg'], 45))
# plt.imshow(ImageGrayScaled(images_query['face_query.jpg']), cmap="gray")
plt.imshow(images_query['face_query.jpg'])
# plt.savefig('face_query' + '.png')

# Histogramme de couleurs

Fonction pour calculer l'histogramme de couleur.

In [ ]:
# Développé par l'équipe (Louis-Philippe et Cyrille)
def PlotHistogram(blue, green, red, imageName):
    plt.figure()
    plt.title("Histogramme de couleur de l'image " + imageName)

    plt.hist(blue, bins=255, color = "blue", alpha = 0.33, density=True)
    plt.hist(green, bins=255, color = "green", alpha = 0.33, density=True)
    plt.hist(red, bins=255, color = "red", alpha = 0.33, density=True)

    plt.xlabel('valeurs des pixels')
    plt.ylabel('Probabilité')
    plt.grid()
    plt.xlim([0,256])
    # Y_max = (np.concatenate((blue,green,red), axis=None)).max()
    # plt.ylim([0, Y_max])
    plt.show()
    plt.savefig('imageName' + '.png')

def ColorHistogram(image, imageName):
    #image = cv.imread(imageName)

    blue  = image[:,:,0].flatten()
    green = image[:,:,1].flatten()
    red   = image[:,:,2].flatten()

    PlotHistogram(blue, green, red, imageName)


##########################################
# Basé sur le code du Github de Gabilodeau 
# Source : https://github.com/gabilodeau/INF6804/blob/master/ColorHisto.ipynb
# Source pour la fonction normalize :
# https://theailearner.com/2019/08/13/comparing-histograms-using-opencv-python/
# ------------------------------------------------------------------------------
def ColorHistogramGabilodeau(image, imageName):
    #image = cv.imread(imageName)

    color = ('b','g','r')
    plt.figure()
    for i, col in enumerate(color):
        hist = cv.calcHist([image], [i], None, [8], [0,256])
        #cv.normalize(hist, hist, alpha=0, beta=1, norm_type=cv.NORM_MINMAX);
        hist = cv.normalize(hist, hist).flatten()
        plt.plot(hist, color = col)
    
    plt.grid()
    plt.title("Histogramme de couleur de l'image " + imageName)
    plt.xlim([0,8])    
    plt.show()

Présentation des résultats des histogramme de couleur pour les images query

In [ ]:
# Show Color histogram of the airplane_query
ColorHistogram(images_query['airplane_query.jpg'], 'airplane_query.jpg')

# Show Color histogram of the strawberry_query
ColorHistogram(images_query['strawberry_query.jpg'], 'strawberry_query.jpg')
ColorHistogramGabilodeau(images_query['strawberry_query.jpg'], 'strawberry_query.jpg')
# Show Color histogram of the ball_query
ColorHistogramGabilodeau(images_query['ball_query.jpg'], 'ball_query.jpg')

# Show Color histogram of the lotus_query
ColorHistogramGabilodeau(images_query['lotus_query.jpg'], 'lotus_query.jpg')

# Show Color histogram of the car_query
ColorHistogramGabilodeau(images_query['car_query.jpg'], 'car_query.jpg')

# Show Color histogram of the cat_query
ColorHistogramGabilodeau(images_query['cat_query.jpg'], 'cat_query.jpg')

# Show Color histogram of the face_query
ColorHistogramGabilodeau(images_query['face_query.jpg'], 'face_query.jpg')

# Show Color histogram of the dolphin_query
ColorHistogramGabilodeau(images_query['dolphin_query.jpg'], 'dolphin_query.jpg')

# Histogramme de gradients orientés HOG

In [ ]:
def Gradient(image, visualize=True):
    # image     = cv.imread(imageName)
    grayImage = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    laplacian = cv.Laplacian(grayImage, cv.CV_64F)

    if visualize is True:
        plt.figure()
        plt.axis("off")
        plt.title("Laplacian")
        plt.imshow(laplacian, cmap="gray")
    else:
        return laplacian 

# Source :
# https://www.thepythoncode.com/article/hog-feature-extraction-in-python#:~:text=The%20Histogram%20of%20Oriented%20Gradients%20%28HOG%29%20is%20a,portion%20of%20an%20image%20or%20region%20of%20interest.
# -----------------------------------------------------------------------------
def HogHistogramm(img , visualize=True):
    # resizing image
    # resized_img = resize(img, (img.shape[0]*4, img.shape[1]*4))
    resized_img = resize(img, (320*4, 280*4))
    
    #creating hog feature
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, multichannel=True)
    if visualize is True:
        plt.figure()
        plt.axis("off")
        #plt.title("Histogramme de gradients orientés HOG " + imageName)
        plt.imshow(hog_image, cmap="gray")
    else:
        return fd


In [ ]:
## test######################################
# Gradient(images_query['airplane_query.jpg'])
HogHistogramm(images_query['airplane_query.jpg'])
HogHistogramm(images_query['strawberry_query.jpg'])
#############################################

# Function Comparaison Query Database (Color Histogram - hog)

In [ ]:
# source :
# https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/#download-the-code
# ------------------------------------------------------------------------------

def CompareHistWithDatabase(name_query , hist_query , index_data, method, resultImageName,
                            reverse=True, hog=False):

    # Declaration of the dictionary result to hold the computation.
    results = dict()
    phrase  = ""

    # Fill the result dictionary with the histogram distance computation
    for (k, hist) in index_data.items():
        # HOG calculation similarities
        if hog == True:
            d = method(hist_query, hist)

        # Color histogram calculation similarities
        else:
            # Airplane works bette with HISTCMP_BHATTACHARYYA
            if name_query == 'airplane_query.jpg':
                method  = cv.HISTCMP_BHATTACHARYYA
                reverse = False

            # Adding a case for the strawberry query image
            if name_query == 'strawberry_query.jpg':
                #range in histograme from 200 to 250
                n,  temp, temp1 = plt.hist(hist[0])
                n1, temp, temp1 = plt.hist(hist[1])
                n2, temp, temp1 = plt.hist(hist[2])
                #check if there is no red and green probability between[6,7]
                #because the hist expand between 0 to 7
                if n[7] > 0.01 and n[7]  < 0.9 and  n1[7] < 0.01 and n2[7] < 0.01 :
                    d = cv.compareHist(hist_query, hist, method)
                    results[k] = d
                else:
                    d = 0.000
                    results[k] = d
                
                continue
            
            d = cv.compareHist(hist_query, hist, method)

        results[k] = d

    # END for loop ############################################################


    # HOG case, sorting the result
    if hog == True :
        phrase  = "Result HOG descriptor"
        results = sorted([(v, k) for (k, v) in results.items()])

    # Color histogram case, sorting the result
    else:
        phrase = "Result Color histogram"
        results = sorted([(v, k) for (k, v) in results.items()], reverse = reverse)

    # Printing the image query
    plt.figure(name_query)
    plt.imshow(images_query[name_query])
    plt.axis("off")
    # plt.savefig(name_query[:-4] + ".png")
    plt.show()

    # Printing the result
    fig = plt.figure(phrase)
    for (i, (v, k)) in enumerate(results):
        # Show only 5 images 
        if i == 5:
            break
        elif v !=0 :
            ax = fig.add_subplot(2, 3, i + 1 )
            ax.set_title("%s:\n %.2f" % (k, v))
            plt.imshow(images_data[k])
            plt.axis("off")

    fig.tight_layout()
    plt.savefig(resultImageName + name_query[:-4] + ".png")
    plt.show()
    

# Comparaison des histogrammes de couleur des images query avec la database 

## **non altérées**

In [ ]:
# Source :
# https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/#download-the-code
# ------------------------------------------------------------------


# Create a dictionary to hold the color histogram of the pictures
ColorHist_query = {}
ColorHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    hist = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv.normalize(hist, hist).flatten()
    ColorHist_query[name] = hist

for name, image in images_data.items() :
    hist = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist = cv.normalize(hist, hist).flatten()
    ColorHist_data[name] = hist

# Different method to compare the color histograms
method  = cv.HISTCMP_CORREL # This is the best for face recognition 
# method  = cv.HISTCMP_CHISQR
# method  = cv.HISTCMP_CHISQR_ALT # The worst
# method  = cv.HISTCMP_INTERSECT
# method  = cv.HISTCMP_BHATTACHARYYA
# reverse = False
reverse = True

# Comparing the color histogram of the query image with the dataset images
for name, hist in ColorHist_query.items():
    CompareHistWithDatabase(name, hist, ColorHist_data, method, "Result_Color_", reverse)


## Cas utilisation 1 : **translation** 

In [ ]:
# Source :
# https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/#download-the-code
# ------------------------------------------------------------------

# Create a dictionary to hold the color histogram of the pictures
ColorHist_query = {}
ColorHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageTranslation(image)
    hist  = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist  = cv.normalize(hist, hist).flatten()
    ColorHist_query[name] = hist

for name, image in images_data.items() :
    hist = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist = cv.normalize(hist, hist).flatten()
    ColorHist_data[name] = hist

# Different method to compare the color histograms
method  = cv.HISTCMP_CORREL # This is the best for face recognition 
# method  = cv.HISTCMP_CHISQR
# method  = cv.HISTCMP_CHISQR_ALT # The worst
# method  = cv.HISTCMP_INTERSECT
# method  = cv.HISTCMP_BHATTACHARYYA
# reverse = False
reverse = True

# Comparing the color histogram of the query image with the dataset images
for name, hist in ColorHist_query.items():
    CompareHistWithDatabase(name, hist, ColorHist_data, method, "Result_Color_Translation_", reverse)


## Cas d'utilisation 2: **Gaussian Noice**

In [ ]:
# Source :
# https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/#download-the-code
# ------------------------------------------------------------------

# Create a dictionary to hold the color histogram of the pictures
ColorHist_query = {}
ColorHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageGaussianNoise(image, 0 ,0.1)
    hist  = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist  = cv.normalize(hist, hist).flatten()
    ColorHist_query[name] = hist

for name, image in images_data.items() :
    hist = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist = cv.normalize(hist, hist).flatten()
    ColorHist_data[name] = hist

# Different method to compare the color histograms
method  = cv.HISTCMP_CORREL # This is the best for face recognition 
# method  = cv.HISTCMP_CHISQR
# method  = cv.HISTCMP_CHISQR_ALT # The worst
# method  = cv.HISTCMP_INTERSECT
# method  = cv.HISTCMP_BHATTACHARYYA
# reverse = False
reverse = True

# Comparing the color histogram of the query image with the dataset images
for name, hist in ColorHist_query.items():
    CompareHistWithDatabase(name, hist, ColorHist_data, method, "Result_Color_Noice_", reverse)


## Cas d'utilisation 3 : **Rotation**

In [ ]:
# Source :
# https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/#download-the-code
# ------------------------------------------------------------------

# Create a dictionary to hold the color histogram of the pictures
ColorHist_query = {}
ColorHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageRotation(image, 45)
    hist  = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist  = cv.normalize(hist, hist).flatten()
    ColorHist_query[name] = hist

for name, image in images_data.items() :
    hist = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist = cv.normalize(hist, hist).flatten()
    ColorHist_data[name] = hist

# Different method to compare the color histograms
method  = cv.HISTCMP_CORREL # This is the best for face recognition 
# method  = cv.HISTCMP_CHISQR
# method  = cv.HISTCMP_CHISQR_ALT # The worst
# method  = cv.HISTCMP_INTERSECT
# method  = cv.HISTCMP_BHATTACHARYYA
# reverse = False
reverse = True

# Comparing the color histogram of the query image with the dataset images
for name, hist in ColorHist_query.items():
    CompareHistWithDatabase(name, hist, ColorHist_data, method, "Result_Color_Rotation_", reverse)


## Cas d'utilisation 4 : **GrayScaled**

In [ ]:
# Source :
# https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/#download-the-code
# ------------------------------------------------------------------

# Create a dictionary to hold the color histogram of the pictures
ColorHist_query = {}
ColorHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageGrayScaled(image)
    hist  = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist  = cv.normalize(hist, hist).flatten()
    ColorHist_query[name] = hist

for name, image in images_data.items() :
    hist = cv.calcHist(image, [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist = cv.normalize(hist, hist).flatten()
    ColorHist_data[name] = hist

# Different method to compare the color histograms
method  = cv.HISTCMP_CORREL # This is the best for face recognition 
# method  = cv.HISTCMP_CHISQR
# method  = cv.HISTCMP_CHISQR_ALT # The worst
# method  = cv.HISTCMP_INTERSECT
# method  = cv.HISTCMP_BHATTACHARYYA
# reverse = False
reverse = True

# Comparing the color histogram of the query image with the dataset images
for name, hist in ColorHist_query.items():
    CompareHistWithDatabase(name, hist, ColorHist_data, method, "Result_Color_Gray_", reverse)


# Comparaison des histogrammes de gradient orienté des images query avec la Database (HOG)


## **Non altérées**

In [ ]:
# Source :
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
# https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#
# -----------------------------------------------------------------------------

# Create a dictionary to hold the hog histogram of the pictures
HogHist_query = {}
HogHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    hist = HogHistogramm(image, visualize=False)
    HogHist_query[name] = hist

for name, image in images_data.items() :
    hist = HogHistogramm(image, visualize=False)
    HogHist_data[name] = hist

method = dist.euclidean # Best method
# method = dist.cityblock
# method = dist.chebyshev
# method = dist.cosine
# mothod = dist.hamming

# Comparing the color histogram of the query image with the dataset images
for name, hist in HogHist_query.items():
    CompareHistWithDatabase(name, hist, HogHist_data, method, "Result_HOG_", reverse=True, hog=True)



## Cas d'utilisation 1 : **Translation**

In [ ]:
# Source :
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
# https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#
# -----------------------------------------------------------------------------

# Create a dictionary to hold the hog histogram of the pictures
HogHist_query = {}
HogHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageTranslation(image)
    hist  = HogHistogramm(image, visualize=False)
    HogHist_query[name] = hist

for name, image in images_data.items() :
    hist = HogHistogramm(image, visualize=False)
    HogHist_data[name] = hist

method = dist.euclidean # Best method
# method = dist.cityblock
# method = dist.chebyshev
# method = dist.cosine
# mothod = dist.hamming

# Comparing the color histogram of the query image with the dataset images
for name, hist in HogHist_query.items():
    CompareHistWithDatabase(name, hist, HogHist_data, method, "Result_HOG_Translation_", reverse=True, hog=True)


## Cas d'utilisation 2 : **Gaussian Noice**

In [ ]:
# Source :
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
# https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#
# -----------------------------------------------------------------------------

# Create a dictionary to hold the hog histogram of the pictures
HogHist_query = {}
HogHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageGaussianNoise(image, 0, 0.1)
    hist  = HogHistogramm(image, visualize=False)
    HogHist_query[name] = hist

for name, image in images_data.items() :
    hist = HogHistogramm(image, visualize=False)
    HogHist_data[name] = hist

method = dist.euclidean # Best method
# method = dist.cityblock
# method = dist.chebyshev
# method = dist.cosine
# mothod = dist.hamming

# Comparing the color histogram of the query image with the dataset images
for name, hist in HogHist_query.items():
    CompareHistWithDatabase(name, hist, HogHist_data, method, "Result_HOG_Noice_", reverse=True, hog=True)


## Cas d'utilisation 3 : **Rotation**

In [ ]:
# Source :
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
# https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#
# -----------------------------------------------------------------------------

# Create a dictionary to hold the hog histogram of the pictures
HogHist_query = {}
HogHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageRotation(image, 45)
    hist  = HogHistogramm(image, visualize=False)
    HogHist_query[name] = hist

for name, image in images_data.items() :
    hist = HogHistogramm(image, visualize=False)
    HogHist_data[name] = hist

method = dist.euclidean # Best method
# method = dist.cityblock
# method = dist.chebyshev
# method = dist.cosine
# mothod = dist.hamming

# Comparing the color histogram of the query image with the dataset images
for name, hist in HogHist_query.items():
    CompareHistWithDatabase(name, hist, HogHist_data, method, "Result_HOG_Rotation_", reverse=True, hog=True)


## Cas d'utilisation 4 : **GrayScaled**

In [ ]:
# Source :
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
# https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#
# -----------------------------------------------------------------------------

# Create a dictionary to hold the hog histogram of the pictures
HogHist_query = {}
HogHist_data  = {}

# Fill the dictionary created previously
for name, image in images_query.items() :
    image = ImageGrayScaled(image)
    hist  = HogHistogramm(image, visualize=False)
    HogHist_query[name] = hist

for name, image in images_data.items() :
    hist = HogHistogramm(image, visualize=False)
    HogHist_data[name] = hist

method = dist.euclidean # Best method
# method = dist.cityblock
# method = dist.chebyshev
# method = dist.cosine
# mothod = dist.hamming

# Comparing the color histogram of the query image with the dataset images
for name, hist in HogHist_query.items():
    CompareHistWithDatabase(name, hist, HogHist_data, method, "Result_HOG_Gray_", reverse=True, hog=True)
